# Chart and map urbanization data

In [ ]:
import sys
import os
import multiprocessing
import boto3

import pandas as pd
import geopandas as gpd

# Import raster helpers
sys.path.insert(0, "../../../../gostrocks/src")
from GOSTrocks.misc import tPrint

# Import GOST urban functions
sys.path.append("../../../src")

# Import local functions
# from novelUrbanization import *

%load_ext autoreload
%autoreload 2

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty"
db_prefix = "AAPPC/Delineations"
pop_lyr = "cpo15"

out_folder = "C:/WBG/Work/Projects/Novel_Urban"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [ ]:
# Get all output raster files for the selected population layer
region = "us-east-1"
s3client = boto3.client("s3", region_name=region, verify=False)

# Loop through the S3 bucket and get all the file keys
more_results = True
try:
    del token  # noqa
except Exception:
    pass
loops = 0

all_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket,
            ContinuationToken=token,  # noqa
            Prefix=prefix,  # noqa
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        x = res["Key"]
        if (x.endswith(".tif")) and (pop_lyr in x):
            all_res.append(x)

In [ ]:
country_res = {}
for cFile in all_res:
    iso3 = os.path.basename(cFile)[:3]
    try:
        country_res[iso3].append(cFile)
    except:
        country_res[iso3] = [cFile]

In [ ]:
# Identify all the datasets for each dartboard dataset


In [ ]:
# Generate comparison of DOU gpo and DB cpo15
iso3 = "AGO"
country_folder = os.path.join(base_folder, f"{iso3}_URBAN_DATA_new_naming")
urb = urban_data(iso3, country_folder, aapc_folder)
comboRes = urb.generate_combo_layer(pop_type=[dou_pop, db_pop], debug=True)
if comboRes:
    if not os.path.exists(agg_folder):
        os.makedirs(agg_folder)
    urb.write_results(comboRes, agg_folder, dbhd="co")
tPrint(iso3)

In [ ]:
def multiP(iso3):
    dou_pop = "gpo"
    db_pop = "cpo15"
    country_folder = os.path.join(base_folder, f"{iso3}_URBAN_DATA_new_naming")
    urb = urban_data(iso3, country_folder, aapc_folder)
    comboRes = urb.generate_combo_layer(pop_type=[dou_pop, db_pop], debug=True)
    if comboRes:
        agg_folder = f"/home/wb411133/data/Projects/MR_Novel_Urbanization/Aggregate_Data/DOU{dou_pop}_DB{db_pop}"
        urb.write_results(comboRes, agg_folder)
    tPrint(iso3)

In [ ]:
with multiprocessing.Pool(len(processed_iso3)) as mp:
    mp.map(multiP, processed_iso3)

In [ ]:
sum_files = []
bin_files = []

for root, dirs, files in os.walk(agg_folder):
    for f in files:
        if f.endswith("sum_co.tif"):
            sum_files.append(os.path.join(root, f))
        if f.endswith("binary_co.tif"):
            bin_files.append(os.path.join(root, f))

In [ ]:
vrt_options = gdal.BuildVRTOptions(resampleAlg="cubic", addAlpha=True)
pop_layer = "cpo"
my_vrt = gdal.BuildVRT(
    f"DOU{dou_pop}_DB{db_pop}_sum.vrt", sum_files, options=vrt_options
)
my_vrt = None
my_vrt = gdal.BuildVRT(
    f"DOU{dou_pop}_DB{db_pop}_binary.vrt", bin_files, options=vrt_options
)
my_vrt = None

# Kappa comparison

In [ ]:
iso3 = "BGD"
country_folder = os.path.join(base_folder, f"{iso3}_URBAN_DATA_new_naming")
urb = urban_data(iso3, country_folder, aapc_folder)
comboRes = urb.generate_combo_layer(pop_type=pop_layer, debug=True)
res = urb.jaccard_index(pop_type=pop_layer)
res

In [ ]:
def jaccardP(iso3, debug=False):
    country_folder = os.path.join(base_folder, f"{iso3}_URBAN_DATA_new_naming")
    urb = urban_data(iso3, country_folder, aapc_folder)
    if not debug:
        try:
            comboRes = urb.generate_combo_layer(pop_type=[dou_pop, db_pop])
            res = urb.jaccard_index(pop_type=[dou_pop, db_pop])
        except:
            res = {"urb_jaccard": -1, "hd_jaccard": -1}
    else:
        comboRes = urb.generate_combo_layer(pop_type=[dou_pop, db_pop])
        res = urb.jaccard_index(pop_type=[dou_pop, db_pop])
    tPrint(f"{iso3}: {res}")
    return {iso3: res}


jaccardP("NAM", True)

"""
with multiprocessing.Pool(len(processed_iso3)) as mp:
    all_jaccard = mp.map(jaccardP, processed_iso3)

"""

In [ ]:
res = {}
for cntry in all_jaccard:
    cISO = list(cntry.keys())[0]
    res[cISO] = cntry[cISO]
res = pd.DataFrame(res).transpose()
res = res.sort_values("urb_jaccard", ascending=False)
res

In [ ]:
# Attach geometry to the res
admin0_polys = gpd.read_file("/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp")
res["ISO3"] = res.index

In [ ]:
temp_res = res.loc[res["hd_jaccard"] > 0]
temp_res = temp_res.merge(admin0_polys, on="ISO3")
temp_res = gpd.GeoDataFrame(temp_res, geometry="geometry", crs=4326)
temp_res.to_file(f"{agg_folder}_national_jaccard.shp")

In [ ]:
temp_res["ISO3"].values

In [ ]:
# Create subset of GHS UCDB
ucdb_file = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
ucdb = gpd.read_file(ucdb_file)
ucdb = ucdb.loc[ucdb["CTR_MN_ISO"].isin(temp_res["ISO3"].values)]

In [ ]:
ucdb.shape

In [ ]:
all_grps = []
cnt_size = 2
for idx, grp in ucdb.groupby("CTR_MN_ISO"):
    grp = grp.sort_values("P15", ascending=False)
    if grp.shape[0] > cnt_size:
        all_grps.append(grp.iloc[0:cnt_size, :])
    else:
        all_grps.append(grp)

In [ ]:
sel_res = pd.concat(all_grps)
sel_res.shape

In [ ]:
sel_res.to_file(f"{agg_folder}_select_cities.geojson", driver="GeoJSON")

In [ ]:
f"{agg_folder}_select_cities.geojson"